In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
metadata = pd.read_csv(
    "/Users/michael/Data/Luke_terrace_experiment/General_data/Plates_1_to_4_metadata_merged_luke.csv",
    index_col=0,
)
metadata = metadata.drop(
    columns=[
        "arb.sort",
        "sample-id",
        "Ambiguous Unstranded",
        "Ambiguous Forward",
        "Multimapping",
        "Unmapped Over Mapped",
    ]
)
metadata["Date and Time"] = metadata["date"] + " " + metadata["time"]
luke_time_data_format = "%-m/%-d/%y %-H:%-M"
metadata["Date and Time"] = pd.to_datetime(
    metadata["Date and Time"], format=luke_time_data_format
)

microbiome_abundance = pd.read_csv(
    "/Users/michael/Data/Luke_terrace_experiment/Microbiome/lic2024_16S_rab.csv"
)
long_term_microbiome = microbiome_abundance.merge(
    metadata[["sampID", "Experiment Type"]], left_on="plantID", right_on="sampID"
)
long_term_microbiome

,Sample,sample.type,timepoint,date,time,platename,daysincestart,plantID,Kingdom,Phylum,Class,Order,Family,Genus,Species,AbundR100,sampID,Experiment Type
0,LIC522,plant,c1_t11,2/11/24,12:00,LICCIRC_02,102,LIC522,d__Bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales_A_650611,Pseudomonadaceae,Pseudomonas_E_647464,Pseudomonas_E_647464 viridiflava,79.1,LIC522,Circadian Experiment 1
1,LIC063,plant,t06,11/28/23,8:00,LIC_01,27,LIC063,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae_486827,Sphingomonas_L_486704,Sphingomonas_L_486704 sp000786205,66.1,LIC063,Long Term
2,LIC159,plant,t14,1/23/24,8:00,LIC_02,83,LIC159,d__Bacteria,Cyanobacteriota,Cyanobacteriia,Cyanobacteriales,Coleofasciculaceae,Caldora,Caldora sp010672925,60.5,LIC159,Long Term
3,LIC452,plant,c1_t06,2/10/24,16:00,LICCIRC_01,101,LIC452,d__Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae_A_595421,Massilia_574544,Massilia atriviolacea,59.8,LIC452,Circadian Experiment 1
4,LIC306,plant,t26,3/8/24,8:00,LIC_04,128,LIC306,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae_486827,Sphingomonas_L_486704,Sphingomonas_L_486704 sp000786205,57.2,LIC306,Long Term
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12281,LIC122,plant,t11,12/15/23,8:00,LIC_02,44,LIC122,d__Bacteria,Actinomycetota,Actinomycetes,Propionibacteriales,Nocardioidaceae,Nocardioides_A_392796,Nocardioides_A_392796 hankookensis,0.5,LIC122,Long Term
12282,LIC028,plant,t03,11/14/23,8:00,LIC_01,13,LIC028,d__Bacteria,Actinomycetota,Actinomycetes,Mycobacteriales,Geodermatophilaceae,Modestobacter,Modestobacter muralis,0.5,LIC028,Long Term
12283,LIC631,plant,c2_t05,3/2/24,12:00,LICCIRC_03,122,LIC631,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Caulobacterales,Caulobacteraceae,Brevundimonas,Brevundimonas subvibrioides,0.5,LIC631,Circadian Experiment 2
12284,LIC629,plant,c2_t05,3/2/24,12:00,LICCIRC_03,122,LIC629,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Rhizobiales_505101,Rhizobiaceae,Agrobacterium,Agrobacterium larrymoorei,0.5,LIC629,Circadian Experiment 2


In [3]:
soil_samples = microbiome_abundance.loc[microbiome_abundance["sample.type"] == "soil"]

In [58]:
soil_samples.loc[soil_samples["Sample"] == "t27_soil"]

,Sample,sample.type,timepoint,date,time,platename,daysincestart,plantID,Kingdom,Phylum,Class,Order,Family,Genus,Species,AbundR100
1372,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Sphingomonadales,Sphingomonadaceae_486827,Sphingomonas_L_486704,Sphingomonas_L_486704 sp000786205,6.2
1664,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales_B_877923,Flavobacteriaceae,Flavobacterium,Flavobacterium chilense,5.2
2084,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae_A_595421,Hylemonella,Ramlibacter terrae,4.3
2477,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Actinomycetota,Actinomycetes,Mycobacteriales,Geodermatophilaceae,Blastococcus,Blastococcus massiliensis,3.7
2709,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae_A_595421,Paucimonas,Paucimonas lemoignei,3.4
3897,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae_A_595421,Herminiimonas_567389,Herminiimonas arsenicoxydans,2.5
3971,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Rhizobiales_505101,Xanthobacteraceae,Bradyrhizobium_503372,Bradyrhizobium sp000938255,2.4
4061,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Gammaproteobacteria,Burkholderiales,Burkholderiaceae_A_595421,Massilia_574544,Massilia atriviolacea,2.3
4584,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Actinomycetota,Thermoleophilia,Solirubrobacterales,Solirubrobacteraceae,Solirubrobacter,Solirubrobacter soli,2.1
4757,t27_soil,soil,t27,3/12/24,8:00,LIC_04,132,t27_soil,d__Bacteria,Pseudomonadota,Alphaproteobacteria,Rhizobiales_505101,Rhizobiaceae,Neorhizobium_500197,Neorhizobium soli,2.0


In [46]:
genus_sum = (
    soil_samples[["Sample", "Genus", "AbundR100"]].groupby(["Sample", "Genus"]).sum()
)
genus_sum = genus_sum.reset_index()
genus_sum

,Sample,Genus,AbundR100
0,c1_t01_soil,Actinomycetospora,0.9
1,c1_t01_soil,Actinoplanes,2.2
2,c1_t01_soil,Aeromicrobium,0.5
3,c1_t01_soil,Amycolatopsis_D_380379,0.8
4,c1_t01_soil,Aquabacterium_A_592457,0.9
...,...,...,...
2254,t28_soil,UBA5216,1.8
2255,t28_soil,VAYN01,1.4
2256,t28_soil,Variovorax,1.2
2257,t28_soil,Virgisporangium,0.8


In [47]:
soil_samples_meta = soil_samples.drop_duplicates(subset=["Sample", "Genus"])
soil_samples_meta

,Sample,sample.type,timepoint,date,time,platename,daysincestart,plantID,Kingdom,Phylum,Class,Order,Family,Genus,Species,AbundR100
53,c1_t08_soil,soil,c1_t08,2/11/24,0:00,LICCIRC_02,102,c1_t08_soil,d__Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales_B_877923,Flavobacteriaceae,Flavobacterium,Flavobacterium chilense,36.0
63,t23_soil,soil,t23,2/26/24,8:00,LIC_03,117,t23_soil,d__Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales_B_877923,Flavobacteriaceae,Flavobacterium,Flavobacterium chilense,34.5
104,c1_t03_soil,soil,c1_t03,2/10/24,4:00,LICCIRC_01,101,c1_t03_soil,d__Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales_B_877923,Flavobacteriaceae,Flavobacterium,Flavobacterium chilense,30.9
142,c2_t12_soil,soil,c2_t12,3/3/24,16:00,LICCIRC_04,123,c2_t12_soil,d__Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales_B_877923,Flavobacteriaceae,Flavobacterium,Flavobacterium chilense,27.9
145,c1_t12_soil,soil,c1_t12,2/11/24,16:00,LICCIRC_02,102,c1_t12_soil,d__Bacteria,Bacteroidota,Bacteroidia,Flavobacteriales_B_877923,Flavobacteriaceae,Flavobacterium,Flavobacterium chilense,27.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16862,c1_t09_soil,soil,c1_t09,2/11/24,4:00,LICCIRC_02,102,c1_t09_soil,d__Bacteria,Actinomycetota,Actinomycetes,Mycobacteriales,Pseudonocardiaceae,Pseudonocardia,Pseudonocardia hierapolitana,0.5
16863,c1_t09_soil,soil,c1_t09,2/11/24,4:00,LICCIRC_02,102,c1_t09_soil,d__Bacteria,Actinomycetota,Actinomycetes,Mycobacteriales,Nakamurellaceae,Nakamurella,Nakamurella deserti,0.5
16876,c1_t02_soil,soil,c1_t02,2/10/24,0:00,LICCIRC_01,101,c1_t02_soil,d__Bacteria,Actinomycetota,Actinomycetes,Mycobacteriales,Cryptosporangiaceae,Cryptosporangium,Cryptosporangium arvum,0.5
16877,t07_soil,soil,t07,12/1/23,8:00,LIC_01,30,t07_soil,d__Bacteria,Actinomycetota,Actinomycetes,Propionibacteriales,Kribbellaceae,Kribbella,Kribbella swartbergensis,0.5


In [48]:
genus_sum

,Sample,Genus,AbundR100
0,c1_t01_soil,Actinomycetospora,0.9
1,c1_t01_soil,Actinoplanes,2.2
2,c1_t01_soil,Aeromicrobium,0.5
3,c1_t01_soil,Amycolatopsis_D_380379,0.8
4,c1_t01_soil,Aquabacterium_A_592457,0.9
...,...,...,...
2254,t28_soil,UBA5216,1.8
2255,t28_soil,VAYN01,1.4
2256,t28_soil,Variovorax,1.2
2257,t28_soil,Virgisporangium,0.8


In [49]:
genus_sum_w_tp = genus_sum.merge(
    soil_samples_meta[["Sample", "timepoint", "date", "time", "Genus"]],
    left_on=["Sample", "Genus"],
    right_on=["Sample", "Genus"],
)
all_present_genus = genus_sum_w_tp["Genus"].unique()

In [50]:
genus_sum_w_tp

,Sample,Genus,AbundR100,timepoint,date,time
0,c1_t01_soil,Actinomycetospora,0.9,c1_t01,2/9/24,20:00
1,c1_t01_soil,Actinoplanes,2.2,c1_t01,2/9/24,20:00
2,c1_t01_soil,Aeromicrobium,0.5,c1_t01,2/9/24,20:00
3,c1_t01_soil,Amycolatopsis_D_380379,0.8,c1_t01,2/9/24,20:00
4,c1_t01_soil,Aquabacterium_A_592457,0.9,c1_t01,2/9/24,20:00
...,...,...,...,...,...,...
2254,t28_soil,UBA5216,1.8,t28,3/14/24,8:00
2255,t28_soil,VAYN01,1.4,t28,3/14/24,8:00
2256,t28_soil,Variovorax,1.2,t28,3/14/24,8:00
2257,t28_soil,Virgisporangium,0.8,t28,3/14/24,8:00


In [51]:
empty_df_to_add

,Sample,Genus,AbundR100,timepoint,date,time


In [54]:
## Need to fill in missing genus in every sample first
for sampID in genus_sum_w_tp["Sample"].unique():
    sampid_only_df = genus_sum_w_tp.loc[genus_sum_w_tp["Sample"] == sampID]
    present_genus_in_samp = sampid_only_df["Genus"].unique()
    difference_in_present_bacteria = list(
        set(all_present_genus) - set(present_genus_in_samp)
    )
    current_time_point = sampid_only_df["timepoint"].iloc[0]
    empty_df_to_add = pd.DataFrame(
        columns=["Sample", "Genus", "AbundR100", "timepoint", "date", "time"]
    )
    empty_df_to_add["Genus"] = difference_in_present_bacteria
    empty_df_to_add["Sample"] = sampID
    empty_df_to_add["AbundR100"] = 0
    empty_df_to_add["timepoint"] = current_time_point
    empty_df_to_add["date"] = sampid_only_df["date"].iloc[0]
    empty_df_to_add["time"] = sampid_only_df["time"].iloc[0]
    genus_sum_w_tp = pd.concat([genus_sum_w_tp, empty_df_to_add], axis=0)

In [57]:
genus_sum_w_tp.head(50)

,Sample,Genus,AbundR100,timepoint,date,time
0,c1_t01_soil,Actinomycetospora,0.9,c1_t01,2/9/24,20:00
1,c1_t01_soil,Actinoplanes,2.2,c1_t01,2/9/24,20:00
2,c1_t01_soil,Aeromicrobium,0.5,c1_t01,2/9/24,20:00
3,c1_t01_soil,Amycolatopsis_D_380379,0.8,c1_t01,2/9/24,20:00
4,c1_t01_soil,Aquabacterium_A_592457,0.9,c1_t01,2/9/24,20:00
5,c1_t01_soil,Arthrobacter_E_385777,0.8,c1_t01,2/9/24,20:00
6,c1_t01_soil,Blastococcus,3.6,c1_t01,2/9/24,20:00
7,c1_t01_soil,Bradyrhizobium_503372,2.2,c1_t01,2/9/24,20:00
8,c1_t01_soil,Caballeronia,1.2,c1_t01,2/9/24,20:00
9,c1_t01_soil,Flavobacterium,6.5,c1_t01,2/9/24,20:00
